# rigidregistration: Quick walk through
The code below provides a quick walk-through demonstrating use of the rigidregistration python package.

If you find this code useful in your own research, please cite the associated publication:
"Image registration of low signal-to-noise cryo-STEM data", Ultramicroscopy (2018), DOI: 10.1016/j.ultramic.2018.04.008

### Getting started
In this example, data which is formatted as .tif files are loaded using the tifffile package.  For other file formats common to electron microscopy data (e.g., .dm3, .ser...) we recommend the excellent hyperspy package for i/o handling.  See hyperspy.org.

In [4]:
# Import libraries and functions
import numpy as np
import matplotlib.pyplot as plt
from tifffile import imread
import rigidregistration

%matplotlib

Using matplotlib backend: Qt5Agg


In [23]:
# Load data and instantiate imstack object  

f="sample_data/CdSe_NPs.tif"                # Filepath to data
stack=np.rollaxis(imread(f),0,3)/float(2**16)           # Rearrange axes and normalize data
s=rigidregistration.stackregistration.imstack(stack)    # Instantiage imstack object.
s.getFFTs() 

In [27]:
# Inspect data in preparation for registration

for i in range(5,10):                      # Select which images from the stack to display
    fig,(ax1,ax2)=plt.subplots(1,2)
    ax1.matshow(stack[:,:,i],cmap='gray')
    ax2.matshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(stack[:,:,i])))),cmap='gray',vmin=np.average(np.log(np.abs(np.fft.fft2(stack[:,:,i]))))) 
    ax1.grid(False)
    ax2.grid(False)
    plt.show()

### Fourier masking
A Fourier mask is used to avoid incorrect cross correlations, by weighting more trustworthy information in frequency space more heavily.

In [26]:
# Observe the effects of varying the cutoff frequency, n, to determine the best mask.

masktype="gaussian"

i,j = 5,9                                    # Choose image pair
for n in np.arange(1,6,.5):                  # Select n values to test
    s.makeFourierMask(mask=masktype,n=n)     # Set the selected Fourier mask
    s.show_Fourier_mask(i=i,j=j)             # Display the results

In [29]:
# Choose best mask

masktype="gaussian"
n=2

s.makeFourierMask(mask=masktype,n=n)
s.show_Fourier_mask(i=i,j=j)

### Calculate image shifts
Calculate the relative shifts between all pairs of images from their cross correlations.

In [30]:
s.findImageShifts(findMaxima='pixel',verbose=False);     # Find shifts. 

### Find and correct outliers in shift matrix
The previous step determines the relative shifts between all pairs of images.  Here, any incorrectly calculated shifts -- which may result from noisy, low SNR data -- are identified and corrected.  First, the shift matrix is displayed and inspected.  Next, outliers are identified.  Outliers are then corrected.

In [31]:
# Show Xij and Yij matrices

s.show_Rij()

In [32]:
# Identify outliers

s.get_outliers(threshold=10)              # Set outlier threshhold
s.show_Rij()

In [33]:
# Correct outliers

s.make_corrected_Rij()    # Correct outliers using the transitivity relations
s.show_Rij_c()            # Display the corrected shift matrix

### Calculate average image

To obtain the average image, each image in the stack is shifted by an amount which is calculated from the shift matrix.  The entire, shifted image stack is then averaged. 

Several functions are available for displaying and saving the resulting average image, and for summarizing the processing that's been applied to the data for quick review.

In [34]:
# Create registered image stack and average

s.get_averaged_image()

In [35]:
# Display final image

s.show()

In [17]:
# Display report of registration procedure

s.show_report()

In [18]:
# Save report of registration procedure

s.save_report("sample_data/sample_report.pdf")

In [22]:
# Save the average image

s.save("sample_data/sample_output.tif")